In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os
import cv2

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from eigenface import EigenFaces
from img_preprocessing import preprocess_images


import pickle

In [2]:
def load_images(path=None, resize=(500, 500)):
    if path is None:
        path = os.path.join(os.getcwd(), 'images')

    images = []
    students = []

    for student in os.listdir(path):
        student_path = os.path.join(path, student)
        
        if os.path.isdir(student_path):
        
            for image in os.listdir(student_path):
                image_path = os.path.join(student_path, image)
        
                if os.path.isfile(image_path):
                    image = Image.open(image_path)
                    image = image.convert('L')
                    
                    if image.size != resize:
                        image = image.resize(resize)
                    
                    image_vector = np.array(image, dtype=np.float64).flatten()                    
                        
                    images.append(image_vector)
                    students.append(student)
    
    return np.array(images), np.array(students)

In [3]:
def crop_images(face_classifier, path=None, output_path=None, resize=(250, 250)):        

    if path is None:
        path = os.path.join(os.getcwd(), 'images')
    
    if output_path is None:
        output_path = os.path.join(os.getcwd(), 'preprocessed_images')

        
    os.makedirs(output_path, exist_ok=True)

    
    images = []
    students = []

    for student in os.listdir(path):
        student_path = os.path.join(path, student)                
        
        if os.path.isdir(student_path):   
            os.makedirs(os.path.join(output_path, student), exist_ok=True)         
        
            for image_name in os.listdir(student_path):
                image_path = os.path.join(student_path, image_name)
        
                if os.path.isfile(image_path):
                    image = cv2.imread(image_path)
                    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                    
                    faces = face_classifier.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(40, 40))
                    
                    if len(faces) == 0:                        
                        continue

                    (x, y, w, h) = faces[0]
                    face_roi = gray_image[y:y+h, x:x+w]
                    
                    img = preprocess_images([face_roi], resize=resize)[0]                      

                    images.append(img)
                    students.append(student)

                    output_image_path = os.path.join(output_path, student, image_name)
                    cv2.imwrite(output_image_path, img.reshape(resize) * 255)

    return np.array(images), np.array(students)                                                             

In [4]:
face_detector = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

images, students = crop_images(face_detector)

In [5]:
# images, students = load_images('preprocessed_images', resize=(250, 250))

In [6]:
x_train, x_test, y_train, y_test = train_test_split(images, students, test_size=0.2, random_state=42, stratify=students)

In [7]:
eigenface_model = EigenFaces(num_components=100).fit(x_train, y_train)

In [8]:
with open('models/eigenface_recognizer.pkl', 'wb') as f:
    pickle.dump(eigenface_model, f)

In [9]:
y_pred = eigenface_model.predict(x_test)

In [10]:
accuracy = accuracy_score(y_test, y_pred)

In [11]:
accuracy

0.9905660377358491

In [12]:
y_t_p = eigenface_model.predict(x_train)


In [13]:
accuracy_score(y_train, y_t_p)

1.0

In [14]:
x_test[0]

array([0.99843964, 0.99843964, 0.99843964, ..., 0.77248974, 0.77197589,
       0.77197589])